### Imports

In [1]:
import os, time
import math
import json
import numpy as np
import pandas as pd, lightgbm as lgb
import matplotlib.pyplot as plt

from collections import defaultdict

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, balanced_accuracy_score

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

from lightgbm import LGBMClassifier

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import make_scorer

import optuna

import warnings
warnings.filterwarnings("ignore")  # hide sklearn/pandas warnings

pd.set_option("display.max_columns",None)

/etc/python/sitecustomize.py:117: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  mod = _original_import(name, globals, locals, fromlist, level)


### Helper Functions

In [2]:
def add_row_level_features(data: pd.DataFrame) -> pd.DataFrame:
    df2 = data.copy()

    # 1) Distance-based
    df2['Genomic_distance'] = (df2['Bin_2'] - df2['Bin_1']).abs()
    df2['Log_distance'] = np.log1p(df2['Genomic_distance'])

    # 2) Transformations of Count
    df2['Log_count'] = np.log1p(df2['Count'])

    # 3) Symmetry / order
    df2['Bin_min'] = df2[['Bin_1','Bin_2']].min(axis=1)
    df2['Bin_max'] = df2[['Bin_1','Bin_2']].max(axis=1)
    df2['Is_forward'] = (df2['Bin_1'] <= df2['Bin_2']).astype(int)
    df2['Midpoint'] = (df2['Bin_1'] + df2['Bin_2']) / 2.0

    # 4) Neighborhood / marginal features
    # Compute marginal sums per (Chr_name, Bin)
    # We'll do this efficiently by stacking
    tmp1 = df2[['Chr_name','Bin_1','Count']].rename(columns={'Bin_1':'Bin'})
    tmp2 = df2[['Chr_name','Bin_2','Count']].rename(columns={'Bin_2':'Bin'})
    # Counts #contacts for a bin with any other bin
    # Cell agnostic
    tmp = pd.concat([tmp1, tmp2], axis=0, ignore_index=True)
    bin_marginal = tmp.groupby(['Chr_name','Bin'])['Count'].sum().rename('Bin_total').reset_index()
    df2 = df2.merge(bin_marginal.rename(columns={'Bin':'Bin_1','Bin_total':'Bin1_total'}),
                    on=['Chr_name','Bin_1'], how='left')
    df2 = df2.merge(bin_marginal.rename(columns={'Bin':'Bin_2','Bin_total':'Bin2_total'}),
                    on=['Chr_name','Bin_2'], how='left')

    df2['Bin_total_min'] = df2[['Bin1_total','Bin2_total']].min(axis=1)
    df2['Bin_total_max'] = df2[['Bin1_total','Bin2_total']].max(axis=1)
    df2['Bin_total_sum'] = df2['Bin1_total'] + df2['Bin2_total']
    df2['Bin_total_diff_abs'] = (df2['Bin1_total'] - df2['Bin2_total']).abs()

    # 5) Observed/expected by distance (chromosome-specific)
    # Expected = mean Count at similar distance bucket within chromosome
    dist_bin = (df2['Genomic_distance'] / max(1, df2['Genomic_distance'].quantile(0.95)/50)).astype(int)
    df2['_dist_bin'] = dist_bin
    exp = df2.groupby(['Chr_name','_dist_bin'])['Count'].mean().rename('Expected_by_dist').reset_index()
    df2 = df2.merge(exp, on=['Chr_name','_dist_bin'], how='left')
    df2['Obs_over_exp_dist'] = df2['Count'] / (df2['Expected_by_dist'] + 1e-9)

    # 6) Percentile rank of Count within distance bin
    def pct_rank(group):
        return group.rank(pct=True)
    df2['Count_pct_in_dist_bin'] = df2.groupby(['Chr_name','_dist_bin'])['Count'].transform(pct_rank)

    # 7) Relative positions on chromosome (if you later add chr length, you can normalize)
    # For now, normalize by within-chr max bin seen in data
    chr_max_bin = pd.concat([
        df2.groupby('Chr_name')['Bin_1'].max(),
        df2.groupby('Chr_name')['Bin_2'].max()
    ], axis=1).max(axis=1).rename('Chr_max_bin')
    df2 = df2.merge(chr_max_bin, left_on='Chr_name', right_index=True, how='left')
    df2['Rel_pos_bin1'] = df2['Bin_1'] / (df2['Chr_max_bin'] + 1e-9)
    df2['Rel_pos_bin2'] = df2['Bin_2'] / (df2['Chr_max_bin'] + 1e-9)
    df2['Rel_pos_mid'] = df2['Midpoint'] / (df2['Chr_max_bin'] + 1e-9)

    # Clean up helper columns
    df2 = df2.drop(columns=['_dist_bin'], errors='ignore')

    return df2

# We aggregate row-level engineered features to per-Cell_name features.
# This is often more aligned with the cell-cycle label being per-cell rather than per-contact.

def aggregate_cell_level(df_rowfeat: pd.DataFrame) -> pd.DataFrame:
    # Choose subset of numeric engineered features to aggregate
    feature_cols = [
        'Genomic_distance','Log_distance','Log_count',
        'Bin_total_min','Bin_total_max','Bin_total_sum','Bin_total_diff_abs',
        'Obs_over_exp_dist','Count_pct_in_dist_bin',
        'Rel_pos_bin1','Rel_pos_bin2','Rel_pos_mid',
        'Chr_leading_eig','Chr_entropy'
    ]
    feature_cols = [c for c in feature_cols if c in df_rowfeat.columns]

    # Aggregations
    agg_funcs = {
        'mean':'mean','std':'std','min':'min','max':'max','median':'median'
    }

    grp = df_rowfeat.groupby(['Cell_name','Target_label'])
    pieces = []
    for fname in feature_cols:
        piece = grp[fname].agg(['mean','std','min','max','median'])
        piece.columns = [f"{fname}_{k}" for k in piece.columns]
        pieces.append(piece)
    agg_df = pd.concat(pieces, axis=1)

    # Also add distance-based proportions (short/medium/long)
    tmp = df_rowfeat.copy()
    q1 = tmp['Genomic_distance'].quantile(0.33)
    q2 = tmp['Genomic_distance'].quantile(0.66)
    tmp['dist_bucket'] = np.where(tmp['Genomic_distance']<=q1, 'short',
                           np.where(tmp['Genomic_distance']<=q2, 'medium','long'))
    prop = tmp.pivot_table(index=['Cell_name','Target_label'],
                           columns='dist_bucket', values='Count',
                           aggfunc='sum', fill_value=0)
    prop = prop.div(prop.sum(axis=1)+1e-9, axis=0)
    prop.columns = [f"prop_{c}" for c in prop.columns]
    agg_df = agg_df.join(prop, how='left')

    agg_df = agg_df.reset_index()
    return agg_df


### Data Engineering

In [3]:
# === Parameters ===
# Provide your input CSV path here:
CSV_PATH = 'Nagano_Aggregated.csv'  # <-- CHANGE THIS to your file path

# Optional: sampling (set to None to use all rows)
ROW_SAMPLE_N = None  # e.g., 1_000_000 for speed during dev

# Memory guardrails for spectral/matrix features
MAX_BINS_PER_CHR_FOR_SPECTRAL = 2000   # build matrix only if unique bins per chr <= this
MAX_CONTACTS_PER_CHR_FOR_SPECTRAL = 400000  # skip spectral if too many edges
RANDOM_STATE = 42
N_JOBS = -1  # used where supported


In [5]:
df = pd.read_csv(CSV_PATH)

# Basic types
df['Bin_1'] = pd.to_numeric(df['Bin_1'], errors='coerce')
df['Bin_2'] = pd.to_numeric(df['Bin_2'], errors='coerce')
df['Count'] = pd.to_numeric(df['Count'], errors='coerce')

# Add row level features
df_feat = add_row_level_features(df)
print('Row-level features shape:', df_feat.shape)
df_feat.head()

Row-level features shape: (20214896, 26)


,Target_label,Cell_name,Chr_name,Bin_1,Bin_2,Count,Genomic_distance,Log_distance,Log_count,Bin_min,Bin_max,Is_forward,Midpoint,Bin1_total,Bin2_total,Bin_total_min,Bin_total_max,Bin_total_sum,Bin_total_diff_abs,Expected_by_dist,Obs_over_exp_dist,Count_pct_in_dist_bin,Chr_max_bin,Rel_pos_bin1,Rel_pos_bin2,Rel_pos_mid
0,late_S,late_S_273,chr1,95,95,72,0,0.000000,4.290459,95,95,1,95.0,279412,279412,279412,279412,558824,0,47.049672,1.530298,0.722324,197,0.482234,0.482234,0.482234
1,late_S,late_S_273,chr1,14,14,42,0,0.000000,3.761200,14,14,1,14.0,235870,235870,235870,235870,471740,0,47.049672,0.892674,0.589925,197,0.071066,0.071066,0.071066
2,late_S,late_S_273,chr1,45,84,2,39,3.688879,1.098612,45,84,1,64.5,215029,278594,215029,278594,493623,63565,2.601316,0.768842,0.537894,197,0.228426,0.426396,0.327411
3,late_S,late_S_273,chr1,52,53,6,1,0.693147,1.945910,52,53,1,52.5,271080,187824,187824,271080,458904,83256,47.049672,0.127525,0.173187,197,0.263959,0.269036,0.266497
4,late_S,late_S_273,chr1,159,159,53,0,0.000000,3.988984,159,159,1,159.0,285184,285184,285184,285184,570368,0,47.049672,1.126469,0.632196,197,0.807107,0.807107,0.807107


In [6]:
# Add Cell level aggregate features
cell_df = aggregate_cell_level(df_feat)
print('Cell-level feature shape:', cell_df.shape)
cell_df.head()

Cell-level feature shape: (1171, 65)


,Cell_name,Target_label,Genomic_distance_mean,Genomic_distance_std,Genomic_distance_min,Genomic_distance_max,Genomic_distance_median,Log_distance_mean,Log_distance_std,Log_distance_min,Log_distance_max,Log_distance_median,Log_count_mean,Log_count_std,Log_count_min,Log_count_max,Log_count_median,Bin_total_min_mean,Bin_total_min_std,Bin_total_min_min,Bin_total_min_max,Bin_total_min_median,Bin_total_max_mean,Bin_total_max_std,Bin_total_max_min,Bin_total_max_max,Bin_total_max_median,Bin_total_sum_mean,Bin_total_sum_std,Bin_total_sum_min,Bin_total_sum_max,Bin_total_sum_median,Bin_total_diff_abs_mean,Bin_total_diff_abs_std,Bin_total_diff_abs_min,Bin_total_diff_abs_max,Bin_total_diff_abs_median,Obs_over_exp_dist_mean,Obs_over_exp_dist_std,Obs_over_exp_dist_min,Obs_over_exp_dist_max,Obs_over_exp_dist_median,Count_pct_in_dist_bin_mean,Count_pct_in_dist_bin_std,Count_pct_in_dist_bin_min,Count_pct_in_dist_bin_max,Count_pct_in_dist_bin_median,Rel_pos_bin1_mean,Rel_pos_bin1_std,Rel_pos_bin1_min,Rel_pos_bin1_max,Rel_pos_bin1_median,Rel_pos_bin2_mean,Rel_pos_bin2_std,Rel_pos_bin2_min,Rel_pos_bin2_max,Rel_pos_bin2_median,Rel_pos_mid_mean,Rel_pos_mid_std,Rel_pos_mid_min,Rel_pos_mid_max,Rel_pos_mid_median,prop_long,prop_medium,prop_short
0,G1_1,G1,13.573416,19.613125,0,152,3.0,1.733732,1.434151,0.0,5.030438,1.386294,1.447007,0.910755,0.693147,4.605170,1.098612,242084.917076,54971.445031,202,866408,242555.0,275830.012893,51198.188958,695,866408,278967.0,517914.929969,97894.553207,1390,1732816,521317.0,33745.095817,41266.028343,0,644563,21461.0,0.442431,0.339843,0.017152,3.651586,0.372153,0.321826,0.204430,0.013574,0.977653,0.226512,0.476313,0.270966,0.015228,1.0,0.465116,0.575340,0.271124,0.015228,1.0,0.590164,0.525826,0.261380,0.015228,1.0,0.527778,0.080471,0.064133,0.855396
1,G1_10,G1,14.334655,14.859562,0,138,10.0,2.153037,1.206297,0.0,4.934474,2.397895,1.571710,1.050115,0.693147,5.579730,1.098612,237703.988320,53912.039855,16,866408,237927.0,281042.945045,50360.343149,658,866408,284293.0,518746.933365,93799.776315,1316,1732816,520250.0,43338.956724,45683.108151,0,676341,31768.0,0.879588,0.724853,0.017152,6.729479,0.703813,0.477453,0.267246,0.013574,0.998679,0.504495,0.475761,0.261822,0.015228,1.0,0.475904,0.581675,0.265884,0.015228,1.0,0.593548,0.528718,0.257791,0.015228,1.0,0.538071,0.095503,0.096534,0.807964
2,G1_100,G1,14.258807,12.846144,0,119,12.0,2.240374,1.126506,0.0,4.787492,2.564949,1.432659,0.950612,0.693147,5.736572,1.098612,237269.247565,52762.832294,202,866408,237139.0,285076.002793,47915.722861,3271,866408,289395.0,522345.250358,89648.355457,6542,1732816,524342.0,47806.755228,46074.269412,0,676341,36528.0,0.811166,0.653210,0.017152,7.057525,0.674723,0.454343,0.260870,0.013574,0.999295,0.490453,0.472324,0.261037,0.015228,1.0,0.473684,0.579220,0.261960,0.015228,1.0,0.595420,0.525772,0.256526,0.015228,1.0,0.536184,0.125605,0.116094,0.758300
3,G1_101,G1,15.851466,15.682642,0,143,12.0,2.272627,1.190906,0.0,4.969813,2.564949,1.608215,1.080224,0.693147,5.988961,1.386294,235970.393580,53398.626857,237,866408,235944.0,281737.187189,48251.797399,658,866408,285542.0,517707.580770,91228.943290,1316,1732816,519277.5,45766.793609,45128.466751,0,674033,34591.5,1.030740,0.879690,0.017152,10.911307,0.737836,0.518441,0.280159,0.013574,0.999977,0.522870,0.467115,0.262684,0.015228,1.0,0.465409,0.583349,0.263072,0.015228,1.0,0.592000,0.525232,0.256384,0.015228,1.0,0.530612,0.101818,0.089761,0.808421
4,G1_102,G1,13.594406,20.913794,0,157,2.0,1.653106,1.468138,0.0,5.062595,1.098612,1.372676,0.835581,0.693147,4.343805,1.098612,243078.080682,55100.264399,646,866408,243314.0,277515.493970,51708.792684,6545,866408,280954.0,520593.574652,97900.114417,13090,1732816,522968.5,34437.413288,42839.752012,0,674033,21381.0,0.364093,0.282027,0.017152,3.217198,0.340168,0.275471,0.180393,0.013574,0.965783,0.215409,0.475513,0.271072,0.015228,1.0,0.464302,0.574079,0.272932,0.015228,1.0,0.600000,0.524796,0.261418,0.015228,1.0,0.530810,0.083050,0.059390,0.857561


# G vs S Classification - Binary

## LGBM at cell level

In [10]:
# Define Train Test Split
X_cell = cell_df.drop(columns=['Cell_name','Target_label'])
y_cell = cell_df['Target_label'].copy()
print(y_cell.value_counts())

# Collapsing different S phases together
y_cell_binary = y_cell.replace({'early_S': 'S', 'mid_S': 'S', 'late_S': 'S'})
print(y_cell_binary.value_counts())

Target_label
late_S     326
early_S    303
G1         280
mid_S      262
Name: count, dtype: int64
Target_label
S     891
G1    280
Name: count, dtype: int64


In [11]:
# Train/test split with stratification (note small N risk)
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cell, y_cell_binary, test_size=0.25, random_state=RANDOM_STATE, stratify=y_cell
)

# 1) Encode labels (fit on train only)
le = LabelEncoder()
yc_train_enc = le.fit_transform(yc_train)
yc_test_enc  = le.transform(yc_test)

# 2) Optional: class imbalance → per-class weights
w = compute_class_weight(class_weight='balanced',
                         classes=np.unique(yc_train_enc),
                         y=yc_train_enc)
class_weight = {int(c): float(wt) for c, wt in zip(np.unique(yc_train_enc), w)}
# LightGBM accepts dict for multiclass (keys = class indices when y is encoded)

# 3) LGBM multiclass classifier
lgbm = LGBMClassifier(
    objective='multiclass',        # multiclass softmax
    num_class=len(le.classes_),    # K
    n_estimators=400,
    learning_rate=0.05,
    num_leaves=63,                 # ~controls tree complexity
    max_depth=-1,                  # no hard cap; you can set 8–12 if overfitting
    min_child_samples=50,
    subsample=0.8,                 # row sampling
    colsample_bytree=0.8,          # feature sampling
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS,
    class_weight=class_weight,     # or set to None to disable weighting
    metric='multi_logloss'
)

# 4) Fit (with early stopping if you like)
lgbm.fit(
    Xc_train, yc_train_enc,
    eval_set=[(Xc_test, yc_test_enc)],
    eval_metric='multi_logloss',
    callbacks=[]
    # for early stopping (LightGBM>=4): callbacks=[lgb.early_stopping(50, verbose=False)]
)

# 5) Predict and map back
yc_pred_enc = lgbm.predict(Xc_test)           # returns class indices
yc_pred = le.inverse_transform(yc_pred_enc)

print('=== Baseline: LightGBM (Cell-Level) ===')
print(classification_report(yc_test, yc_pred, target_names=le.classes_))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9878
[LightGBM] [Info] Number of data points in the train set: 878, number of used features: 50
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

# Baseline LGBM on cell level - Multiclass

In [6]:
# Define Train Test Split
X_cell = cell_df.drop(columns=['Cell_name','Target_label'])
y_cell = cell_df['Target_label'].copy()

# Train/test split with stratification (note small N risk)
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cell, y_cell, test_size=0.25, random_state=RANDOM_STATE, stratify=y_cell
)

# 1) Encode labels (fit on train only)
le = LabelEncoder()
yc_train_enc = le.fit_transform(yc_train)
yc_test_enc  = le.transform(yc_test)

# 2) Optional: class imbalance → per-class weights
w = compute_class_weight(class_weight='balanced',
                         classes=np.unique(yc_train_enc),
                         y=yc_train_enc)
class_weight = {int(c): float(wt) for c, wt in zip(np.unique(yc_train_enc), w)}
# LightGBM accepts dict for multiclass (keys = class indices when y is encoded)

# 3) LGBM multiclass classifier
lgbm = LGBMClassifier(
    objective='multiclass',        # multiclass softmax
    num_class=len(le.classes_),    # K
    n_estimators=400,
    learning_rate=0.05,
    num_leaves=63,                 # ~controls tree complexity
    max_depth=-1,                  # no hard cap; you can set 8–12 if overfitting
    min_child_samples=50,
    subsample=0.8,                 # row sampling
    colsample_bytree=0.8,          # feature sampling
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS,
    class_weight=class_weight,     # or set to None to disable weighting
    metric='multi_logloss'
)

# 4) Fit (with early stopping if you like)
lgbm.fit(
    Xc_train, yc_train_enc,
    eval_set=[(Xc_test, yc_test_enc)],
    eval_metric='multi_logloss',
    callbacks=[]
    # for early stopping (LightGBM>=4): callbacks=[lgb.early_stopping(50, verbose=False)]
)

# 5) Predict and map back
yc_pred_enc = lgbm.predict(Xc_test)           # returns class indices
yc_pred = le.inverse_transform(yc_pred_enc)

print('=== Baseline: LightGBM (Cell-Level) ===')
print(classification_report(yc_test, yc_pred, target_names=le.classes_))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000866 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9878
[LightGBM] [Info] Number of data points in the train set: 878, number of used features: 50
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

#### Iteration 1 — Train on top-N features (from CV gain importances)

In [15]:
def cv_eval(X_in, y_enc, n_splits=5, verbose=True, class_boost=None):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    oof = np.full_like(y_enc, -1)
    imp_gain = np.zeros(X_in.shape[1], dtype=float)
    metrics = []
    for fold, (tr, va) in enumerate(skf.split(X_in, y_enc), 1):
        X_tr, X_va = X_in.iloc[tr], X_in.iloc[va]
        y_tr, y_va = y_enc[tr], y_enc[va]

        w = compute_class_weight(class_weight='balanced', classes=classes_, y=y_tr)
        cw = {int(c): float(wt) for c, wt in zip(classes_, w)}
        # Optional class emphasis (e.g., upweight mid_S)
        if class_boost:
            for cls_name, mult in class_boost.items():
                if cls_name in le.classes_:
                    cw[int(np.where(le.classes_ == cls_name)[0][0])] *= float(mult)

        model = LGBMClassifier(
            objective='multiclass', num_class=n_classes,
            n_estimators=2000, learning_rate=0.03,
            num_leaves=63, max_depth=-1, min_child_samples=50,
            subsample=0.8, colsample_bytree=0.8,
            reg_alpha=0.0, reg_lambda=0.0,
            random_state=RANDOM_STATE, n_jobs=N_JOBS, metric='multi_logloss',
            class_weight=cw,

            verbosity=-1, 
        )
        model.fit(
            X_tr, y_tr, eval_set=[(X_va, y_va)], eval_metric='multi_logloss',
            callbacks=[lgb.early_stopping(50, verbose=False)]
        )

        y_hat = model.predict(X_va)
        oof[va] = y_hat

        # gain importances from the underlying booster
        gain = model.booster_.feature_importance(importance_type='gain')
        imp_gain += gain

        acc  = accuracy_score(y_va, y_hat)
        f1m  = f1_score(y_va, y_hat, average='macro')
        bacc = balanced_accuracy_score(y_va, y_hat)
        metrics.append((acc, f1m, bacc))
        if verbose:
            print(f"Fold {fold}: acc={acc:.3f} macroF1={f1m:.3f} bAcc={bacc:.3f} iter={model.best_iteration_}")

    acc = accuracy_score(y_enc, oof)
    f1m = f1_score(y_enc, oof, average='macro')
    bacc= balanced_accuracy_score(y_enc, oof)
    if verbose:
        print(f"\nOOF: acc={acc:.3f} macroF1={f1m:.3f} bAcc={bacc:.3f}")

    imp_df = pd.DataFrame({'feature': X_in.columns, 'gain': imp_gain / np.maximum(1, imp_gain.sum())})
    imp_df.sort_values('gain', ascending=False, inplace=True)
    return {'oof': oof, 'metrics': metrics, 'oof_acc': acc, 'oof_f1m': f1m, 'oof_bacc': bacc, 'imp': imp_df}

In [12]:
RANDOM_STATE = 42
N_JOBS = -1

X = cell_df.drop(columns=['Cell_name','Target_label'])
y = cell_df['Target_label'].copy()
le = LabelEncoder()
y_enc = le.fit_transform(y)
classes_, n_classes = np.unique(y_enc), len(np.unique(y_enc))


# 1A) Baseline CV importances
res0 = cv_eval(X, y_enc)
imp_df = res0['imp']
print(imp_df.head(20))


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000594 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9912
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 52
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

In [21]:
# 1B) Keep top-N features
for i in [10, 20,30,40,50,60,70]:
    print(f"Iteration for Top {i} features")
    TOP_N = i  # try 40, 60, 80 and pick best
    top_feats = imp_df['feature'].head(TOP_N).tolist()
    res1 = cv_eval(X[top_feats], y_enc)
    print(f"Top-{TOP_N} OOF: acc={res1['oof_acc']:.3f} macroF1={res1['oof_f1m']:.3f} bAcc={res1['oof_bacc']:.3f}")
    print()
    print()
    

Iteration for Top 10 features
Fold 1: acc=0.766 macroF1=0.765 bAcc=0.765 iter=110
Fold 2: acc=0.748 macroF1=0.739 bAcc=0.743 iter=80
Fold 3: acc=0.761 macroF1=0.758 bAcc=0.760 iter=111
Fold 4: acc=0.782 macroF1=0.778 bAcc=0.778 iter=145
Fold 5: acc=0.769 macroF1=0.767 bAcc=0.770 iter=105

OOF: acc=0.765 macroF1=0.762 bAcc=0.763
Top-10 OOF: acc=0.765 macroF1=0.762 bAcc=0.763


Iteration for Top 20 features
Fold 1: acc=0.757 macroF1=0.756 bAcc=0.756 iter=129
Fold 2: acc=0.765 macroF1=0.759 bAcc=0.761 iter=94
Fold 3: acc=0.778 macroF1=0.774 bAcc=0.777 iter=164
Fold 4: acc=0.812 macroF1=0.807 bAcc=0.807 iter=110
Fold 5: acc=0.799 macroF1=0.796 bAcc=0.798 iter=122

OOF: acc=0.782 macroF1=0.779 bAcc=0.780
Top-20 OOF: acc=0.782 macroF1=0.779 bAcc=0.780


Iteration for Top 30 features
Fold 1: acc=0.766 macroF1=0.763 bAcc=0.764 iter=149
Fold 2: acc=0.765 macroF1=0.760 bAcc=0.761 iter=92
Fold 3: acc=0.821 macroF1=0.817 bAcc=0.819 iter=165
Fold 4: acc=0.795 macroF1=0.789 bAcc=0.789 iter=134
Fold 

#### Iteration 2 — Remove multicollinearity (correlation filter within top-N)

In [22]:
def drop_correlated(X_in, importance, corr_thr=0.90, method='spearman'):
    """
    Keep the more important feature when |corr|>thr.
    `importance` is a Series indexed by feature with higher=better.
    """
    corr = X_in.corr(method=method).abs()
    to_drop = set()
    feats = list(X_in.columns)
    # sort pairs by correlation descending to resolve the worst redundancy first
    pairs = [
        (corr.iloc[i, j], feats[i], feats[j])
        for i in range(len(feats)) for j in range(i+1, len(feats))
    ]
    pairs.sort(reverse=True, key=lambda x: x[0])
    for c, f1, f2 in pairs:
        if c <= corr_thr: break
        if f1 in to_drop or f2 in to_drop: continue
        # drop the lower-importance one
        if importance.get(f1, 0.0) >= importance.get(f2, 0.0):
            to_drop.add(f2)
        else:
            to_drop.add(f1)
    kept = [f for f in feats if f not in to_drop]
    return kept, to_drop

In [25]:
# 2A) Apply correlation filter inside top-N
imp_series = imp_df.set_index('feature')['gain']
kept_feats, dropped_feats = drop_correlated(X[top_feats], imp_series, corr_thr=0.60, method='spearman')
print(f"De-correlated: kept={len(kept_feats)} dropped={len(dropped_feats)}")

# 2B) Re-evaluate
res2 = cv_eval(X[kept_feats], y_enc)
print(f"De-correlated OOF: acc={res2['oof_acc']:.3f} macroF1={res2['oof_f1m']:.3f} bAcc={res2['oof_bacc']:.3f}")

De-correlated: kept=51 dropped=12
Fold 1: acc=0.745 macroF1=0.743 bAcc=0.742 iter=140
Fold 2: acc=0.786 macroF1=0.782 bAcc=0.784 iter=151
Fold 3: acc=0.816 macroF1=0.813 bAcc=0.815 iter=139
Fold 4: acc=0.782 macroF1=0.779 bAcc=0.778 iter=133
Fold 5: acc=0.778 macroF1=0.774 bAcc=0.776 iter=146

OOF: acc=0.781 macroF1=0.778 bAcc=0.779
De-correlated OOF: acc=0.781 macroF1=0.778 bAcc=0.779


#### Iteration 3 — Add interaction & summary features (model-agnostic, no leakage)

In [26]:
from itertools import combinations

def gini_row(x):
    v = np.sort(np.abs(x.values.astype(float)))
    if v.sum() == 0: return 0.0
    n = len(v)
    # Gini = (2*sum(i*v_i)/(n*sum(v))) - (n+1)/n
    return (2.0*np.dot(np.arange(1, n+1), v) / (n * v.sum())) - (n+1)/n

def build_summary_block(X_in):
    out = pd.DataFrame(index=X_in.index)
    out['row_mean'] = X_in.mean(axis=1)
    out['row_std']  = X_in.std(axis=1)
    out['row_min']  = X_in.min(axis=1)
    out['row_max']  = X_in.max(axis=1)
    out['row_sum']  = X_in.sum(axis=1)
    out['row_nonzero'] = (X_in != 0).sum(axis=1)
    out['row_p25'] = X_in.quantile(0.25, axis=1)
    out['row_p50'] = X_in.quantile(0.50, axis=1)
    out['row_p75'] = X_in.quantile(0.75, axis=1)
    out['row_gini'] = X_in.apply(gini_row, axis=1)
    return out

def build_interactions(X_in, max_feats=20, make_products=True, make_ratios=True):
    feats = list(X_in.columns[:max_feats])  # take first max_feats
    out = pd.DataFrame(index=X_in.index)
    # products
    if make_products:
        for a, b in combinations(feats, 2):
            out[f'{a}__x__{b}'] = X_in[a].values * X_in[b].values
    # ratios (safe)
    if make_ratios:
        eps = 1e-6
        for a, b in combinations(feats, 2):
            out[f'{a}__over__{b}'] = (X_in[a].values + eps) / (X_in[b].values + eps)
            out[f'{b}__over__{a}'] = (X_in[b].values + eps) / (X_in[a].values + eps)
    return out



In [29]:
# 3A) Build feature blocks (unsupervised transforms -> no leakage)
M = min(20, len(kept_feats))  # cap to avoid explosion
X_base = X[kept_feats].copy()
X_sum  = build_summary_block(X_base)
X_int  = build_interactions(X_base, max_feats=M, make_products=True, make_ratios=True)

# Optional: log1p-transform raw and interaction blocks if counts/heavy-tailed
# X_base = np.log1p(X_base); X_int = np.log1p(X_int)

# Combine
X_aug = pd.concat([X_base, X_sum, X_int], axis=1)

# 3B) Re-evaluate with augmented features
res3 = cv_eval(X_aug, y_enc)
print(f"Augmented OOF: acc={res3['oof_acc']:.3f} macroF1={res3['oof_f1m']:.3f} bAcc={res3['oof_bacc']:.3f}")


Fold 1: acc=0.762 macroF1=0.760 bAcc=0.759 iter=119
Fold 2: acc=0.782 macroF1=0.774 bAcc=0.776 iter=103
Fold 3: acc=0.829 macroF1=0.827 bAcc=0.829 iter=138
Fold 4: acc=0.829 macroF1=0.825 bAcc=0.824 iter=119
Fold 5: acc=0.816 macroF1=0.811 bAcc=0.813 iter=95

OOF: acc=0.804 macroF1=0.800 bAcc=0.800
Augmented OOF: acc=0.804 macroF1=0.800 bAcc=0.800


#### Iteration 4 — Hyperparameter tuning

In [36]:
# ==== helper: quiet fit ====
def quiet_fit(model, *fit_args, **fit_kwargs):
    """Mute stdout/stderr while fitting (LightGBM can be chatty)."""
    with open(os.devnull, "w") as devnull, \
         contextlib.redirect_stdout(devnull), \
         contextlib.redirect_stderr(devnull):
        return model.fit(*fit_args, **fit_kwargs)

# ==== helper: CV evaluation with per-fold class weights & early stopping ====
def cv_eval_lgbm(X_in, y_in, base_params, n_splits=5, class_boost=None, early_stopping_rounds=50):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    oof = np.full_like(y_in, -1)
    fold_rows = []
    feats = list(X_in.columns)

    for fold, (tr, va) in enumerate(skf.split(X_in, y_in), 1):
        X_tr, X_va = X_in.iloc[tr], X_in.iloc[va]
        y_tr, y_va = y_in[tr], y_in[va]

        # per-fold class weights
        w = compute_class_weight('balanced', classes=classes_idx, y=y_tr)
        cw = {int(c): float(wt) for c, wt in zip(classes_idx, w)}
        # optional emphasis (e.g., mid_S)
        if class_boost:
            for cls_name, mult in class_boost.items():
                if cls_name in class_names:
                    idx = int(np.where(np.array(class_names) == cls_name)[0][0])
                    cw[idx] *= float(mult)

        params = dict(base_params)
        params.update({
            'objective': 'multiclass',
            'num_class': n_classes,
            'n_jobs': N_JOBS,
            'random_state': RANDOM_STATE,
            'verbosity': -1,
            'class_weight': cw
        })

        model = LGBMClassifier(**params)

        # early stopping on the fold's validation set
        quiet_fit(
            model,
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric='multi_logloss',
            callbacks=[
                lgb.early_stopping(early_stopping_rounds, verbose=False),
                lgb.log_evaluation(period=0),
            ],
        )

        y_hat = model.predict(X_va)
        oof[va] = y_hat

        fold_rows.append({
            'fold': fold,
            'accuracy': accuracy_score(y_va, y_hat),
            'macro_f1': f1_score(y_va, y_hat, average='macro'),
            'balanced_acc': balanced_accuracy_score(y_va, y_hat),
            'best_iter': getattr(model, 'best_iteration_', None)
        })

    # overall (OOF)
    oof_acc  = accuracy_score(y_in, oof)
    oof_f1m  = f1_score(y_in, oof, average='macro')
    oof_bacc = balanced_accuracy_score(y_in, oof)

    # per-class report (in label names)
    oof_labels = le.inverse_transform(oof)
    rep = classification_report(y, oof_labels, target_names=class_names, output_dict=True)

    # per-class F1 as a DataFrame
    per_class_f1 = pd.Series({cls: rep[cls]['f1-score'] for cls in class_names})

    # confusion matrices
    cm = confusion_matrix(y, oof_labels, labels=class_names)
    cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)

    return {
        'per_fold': pd.DataFrame(fold_rows),
        'oof_acc': oof_acc,
        'oof_f1m': oof_f1m,
        'oof_bacc': oof_bacc,
        'per_class_f1': per_class_f1,
        'cm': pd.DataFrame(cm, index=[f"true_{c}" for c in class_names],
                                columns=[f"pred_{c}" for c in class_names]),
        'cm_norm': pd.DataFrame(cm_norm, index=[f"true_{c}" for c in class_names],
                                      columns=[f"pred_{c}" for c in class_names]),
        'report_dict': rep
    }

In [34]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=60, show_progress_bar=True)

# print("Best macro-F1:", study.best_value)
# print("Best params:", study.best_trial.params)

In [ ]:
# ==== imports ====
import os, contextlib, sys
from itertools import combinations

# ==== data & labels ====
RANDOM_STATE = 42
N_JOBS = -1

# If you built X_aug in previous steps, we'll use it; else fallback to raw features.
try:
    X = X_aug.copy()
    print("Using augmented features X_aug.")
except NameError:
    X = cell_df.drop(columns=['Cell_name','Target_label']).copy()
    print("Using raw features from cell_df.")
y = cell_df['Target_label'].copy()

le = LabelEncoder()
y_enc = le.fit_transform(y)
classes_idx = np.unique(y_enc)
n_classes = len(classes_idx)
class_names = list(le.classes_)



# ==== 1) Baseline CV ====
baseline_params = dict(
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=63,
    max_depth=-1,
    min_child_samples=50,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.0,
    reg_lambda=0.0,
    subsample_freq=1,
    boosting_type='gbdt',
)

print("\n=== Baseline CV evaluation ===")
baseline_res = cv_eval_lgbm(
    X, y_enc, base_params=baseline_params,
    n_splits=5, class_boost=None,  # set {'mid_S': 1.4} if you want emphasis already
    early_stopping_rounds=50
)
print(baseline_res['per_fold'].round(3))
print(f"Baseline OOF: acc={baseline_res['oof_acc']:.3f} "
      f"macroF1={baseline_res['oof_f1m']:.3f} bAcc={baseline_res['oof_bacc']:.3f}")

# ==== 2) RandomizedSearchCV (your param_dist) ====
param_dist = {
    'num_leaves': [31, 63, 127, 255],
    'max_depth': [-1, 8, 10, 12],
    'min_child_samples': [20, 50, 100, 150],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 2.0],
    'reg_lambda': [0.0, 0.5, 1.0, 5.0, 10.0],
    'learning_rate': [0.01, 0.02, 0.05, 0.08],
    'boosting_type': ['gbdt', 'dart']
}

# balanced class weights for search (global, since sklearn CV can't pass fold-wise eval_set easily)
w_global = compute_class_weight('balanced', classes=classes_idx, y=y_enc)
cw_global = {int(c): float(wt) for c, wt in zip(classes_idx, w_global)}
# mid_S emphasis during search (optional; comment out to disable)
if 'mid_S' in class_names:
    mid_idx = int(np.where(np.array(class_names) == 'mid_S')[0][0])
    cw_global[mid_idx] *= 1.4

search_base = LGBMClassifier(
    objective='multiclass',
    num_class=n_classes,
    n_estimators=3000,   # no ES inside sklearn CV; set high and let reg do the work
    subsample_freq=1,
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS,
    class_weight=cw_global,
    verbosity=-1
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
rs = RandomizedSearchCV(
    search_base,
    param_distributions=param_dist,
    n_iter=60,
    cv=cv,
    scoring=make_scorer(f1_score, average='macro'),
    verbose=0,
    n_jobs=N_JOBS,
    refit=True,
    random_state=RANDOM_STATE,
)

print("\n=== Hyperparameter search (RandomizedSearchCV) ===")
# silence any training chatter during the search as well
with open(os.devnull, "w") as devnull, \
     contextlib.redirect_stdout(devnull), \
     contextlib.redirect_stderr(devnull):
    rs.fit(X, y_enc)

best_params = rs.best_params_
best_score = rs.best_score_
print("Best params:", best_params)
print(f"Best CV macro-F1 (search internal): {best_score:.3f}")

# ==== 3) Tuned CV with early stopping & per-fold weights (fair apples-to-apples) ====
tuned_params = dict(baseline_params)   # start from baseline defaults
tuned_params.update(best_params)       # apply search winners
tuned_params.update({'n_estimators': 4000})  # allow ES to find best_iter per fold

print("\n=== Tuned CV evaluation (with ES & per-fold weights) ===")
tuned_res = cv_eval_lgbm(
    X, y_enc, base_params=tuned_params,
    n_splits=5, class_boost={'mid_S': 1.4},   # keep the emphasis if you used it in search
    early_stopping_rounds=100
)
print(tuned_res['per_fold'].round(3))
print(f"Tuned OOF: acc={tuned_res['oof_acc']:.3f} "
      f"macroF1={tuned_res['oof_f1m']:.3f} bAcc={tuned_res['oof_bacc']:.3f}")



Using augmented features X_aug.

=== Baseline CV evaluation ===
   fold  accuracy  macro_f1  balanced_acc  best_iter
0     1     0.766     0.764         0.764        117
1     2     0.795     0.790         0.791         99
2     3     0.812     0.808         0.811        133
3     4     0.816     0.814         0.813        134
4     5     0.799     0.793         0.796        115
Baseline OOF: acc=0.798 macroF1=0.794 bAcc=0.795

=== Hyperparameter search (RandomizedSearchCV) ===


In [ ]:
# ==== 4) Before vs After summary ====
summary = pd.DataFrame({
    'metric': ['OOF Accuracy', 'OOF Macro F1', 'OOF Balanced Acc'],
    'baseline': [baseline_res['oof_acc'], baseline_res['oof_f1m'], baseline_res['oof_bacc']],
    'tuned':    [tuned_res['oof_acc'], tuned_res['oof_f1m'], tuned_res['oof_bacc']]
}).set_index('metric').round(3)
print("\n=== BEFORE vs AFTER (overall) ===")
print(summary)

per_class = pd.DataFrame({
    'baseline_F1': baseline_res['per_class_f1'],
    'tuned_F1':    tuned_res['per_class_f1']
}).loc[class_names].round(3)
print("\n=== BEFORE vs AFTER (per-class F1) ===")
print(per_class)

print("\nConfusion Matrix — Baseline (counts):")
print(baseline_res['cm'])
print("\nConfusion Matrix — Tuned (counts):")
print(tuned_res['cm'])

print("\nConfusion Matrix — Baseline (row-normalized):")
print(baseline_res['cm_norm'].round(3))
print("\nConfusion Matrix — Tuned (row-normalized):")
print(tuned_res['cm_norm'].round(3))
